In [4]:
!pip install PyPDF2 sentence-transformers scikit-learn transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.7 MB/s eta 0:00:00


In [2]:
import os
import gradio as gr
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

# Load vector DB or create it
def load_vector_db():
    if not os.path.exists("vectorstore/index.faiss"):
        print("📄 Creating vector DB from papers...")
        loaders = [
            PyPDFLoader("papers/transformer.pdf"),
            PyPDFLoader("papers/rag.pdf"),
            PyPDFLoader("papers/gpt3.pdf")
        ]
        pages = []
        for loader in loaders:
            pages.extend(loader.load())

        splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        chunks = splitter.split_documents(pages)

        embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        db = FAISS.from_documents(chunks, embeddings)
        db.save_local("vectorstore")
    else:
        embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        db = FAISS.load_local("vectorstore", embeddings)
    return db

# Load DB and model
vector_db = load_vector_db()
retriever = vector_db.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(model_name="gpt-4"),
    retriever=retriever,
    return_source_documents=True
)

# Answer function
def answer_question(query):
    if not query.strip():
        return "Please enter a valid question."
    result = qa_chain({"query": query})
    answer = result["result"]
    sources = "\n\n".join([f"📄 Page: {doc.metadata.get('page', 'N/A')}\n{doc.page_content[:300]}..." for doc in result["source_documents"]])
    return f"🧠 Answer:\n{answer}\n\n📚 Sources:\n{sources}"

# Gradio UI
gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question about the AI papers..."),
    outputs="text",
    title="RAG-based QA System",
    description="Ask questions from 3 AI papers (Transformer, RAG, GPT-3)."
).launch(share=True)


ModuleNotFoundError: Module langchain_community.document_loaders not found. Please install langchain-community to access this module. You can install it using `pip install -U langchain-community`

In [8]:
# Install necessary packages before execution (if not already installed)
# !pip install PyPDF2 sentence-transformers scikit-learn transformers torch

import os
import PyPDF2
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline

# ------------------------------------------
# STEP 1: Load and Preprocess PDF Documents
# ------------------------------------------

def extract_text_from_pdf(pdf_path):
    full_text = ""
    with open(pdf_path, "rb") as f:
        pdf_reader = PyPDF2.PdfReader(f)
        for pg in pdf_reader.pages:
            full_text += pg.extract_text() + "\n"
    return full_text

def split_text_to_chunks(text, chunk_len=500):
    tokens = text.split()
    return [" ".join(tokens[i:i + chunk_len]) for i in range(0, len(tokens), chunk_len)]

def embed_text_chunks(chunks, model_name='all-MiniLM-L6-v2'):
    embedder = SentenceTransformer(model_name)
    embeddings = embedder.encode(chunks, convert_to_numpy=True)
    return embeddings, chunks

# ------------------------------------------
# STEP 2: Similarity Search (Retriever)
# ------------------------------------------

def find_top_chunks(query, chunks, chunk_embeddings, top_k=3, model_name='all-MiniLM-L6-v2'):
    encoder = SentenceTransformer(model_name)
    query_vec = encoder.encode([query], convert_to_numpy=True)
    scores = cosine_similarity(query_vec, chunk_embeddings)[0]
    ranked_indices = np.argsort(scores)[-top_k:][::-1]
    return [(chunks[i], scores[i]) for i in ranked_indices]

# ------------------------------------------
# STEP 3: Generate Answer using Language Model
# ------------------------------------------

def create_answer(query, selected_chunks, gen_model="distilgpt2"):
    context_data = "\n".join(chunk for chunk, _ in selected_chunks)
    prompt_template = f"Question: {query}\nContext: {context_data}\nAnswer:"
    generator = pipeline("text-generation", model=gen_model)
    result = generator(prompt_template, max_length=200, num_return_sequences=1)[0]['generated_text']
    return result.strip(), context_data

# ------------------------------------------
# STEP 4: Source Reporting
# ------------------------------------------

def list_sources(relevant_chunks):
    return [
        f"Snippet {idx+1}: \"{chunk[:100]}...\" (Match: {score:.4f})"
        for idx, (chunk, score) in enumerate(relevant_chunks)
    ]

# ------------------------------------------
# STEP 5: Combined RAG System
# ------------------------------------------

def execute_rag_pipeline(question, pdf_file_paths):
    combined_chunks, all_vecs = [], []

    for doc in pdf_file_paths:
        print(f"📄 Reading: {doc}")
        raw_text = extract_text_from_pdf(doc)
        doc_chunks = split_text_to_chunks(raw_text)
        doc_vecs, _ = embed_text_chunks(doc_chunks)
        combined_chunks.extend(doc_chunks)
        all_vecs.append(doc_vecs)

    final_embeddings = np.vstack(all_vecs)
    top_chunks = find_top_chunks(question, combined_chunks, final_embeddings)
    generated_answer, _ = create_answer(question, top_chunks)
    sources = list_sources(top_chunks)

    return generated_answer, sources

# ------------------------------------------
# Execute with Example Query
# ------------------------------------------

if __name__ == "__main__":
    dataset_dir = "/content/drive/MyDrive/rag/2005.11401v4.pdf"  # Single PDF
    pdf_files = [dataset_dir]  # Treat as single-item list

    question_input = "What are the main components of a RAG model and how do they interact?"

    if not pdf_files:
        print("⚠️ No PDF files found.")
    else:
        response, references = execute_rag_pipeline(question_input, pdf_files)
        print(f"\n🔍 Question: {question_input}")
        print("\n🧠 Answer:\n", response)
        print("\n📚 References:")
        for ref in references:
            print(ref)



📄 Reading: /content/drive/MyDrive/rag/2005.11401v4.pdf


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



🔍 Question: What are the main components of a RAG model and how do they interact?

🧠 Answer:
 Question: What are the main components of a RAG model and how do they interact?
Context: retriever p(zjx)with parameters that returns (top-K truncated) distributions over text passages given a query xand (ii) a generator p(yijx;z;y 1:i 1)parametrized 1Code to run experiments with RAG has been open-sourced as part of the HuggingFace Transform- ers Library [ 66] and can be found at https://github.com/huggingface/transformers/blob/master/ examples/rag/ . An interactive demo of RAG models can be found at https://huggingface.co/rag/ 2 bythat generates a current token based on a context of the previous i 1tokensy1:i 1, the original inputxand a retrieved passage z. To train the retriever and generator end-to-end, we treat the retrieved document as a latent variable. We propose two models that marginalize over the latent documents in different ways to produce a distribution over generated text. I